In [ ]:
# Щас обучаем генерацию
# Есть данные разной длины (от 0 до 100) разных соотношений
# Оставляем по одному соотношению (выбранному случайно) для каждой длины для валидации на низвестных соотношениях
# Отделяем 5-10% данных от каждого набора длин для валидации на известных соотношениях
# 
# 
# Есть куча соотношений
# Эксп-1: префикс yn, ny, nn + два соотношения. Просим сгенерировать yy (фича - сэмплить наборы, где слово по редакшн решио далеко/близко от соотношений для nn)
# Какие проблемы: для коротких длин слова из фригруппы равны соотношению
# Изначально надо намайнить пары соотношений. Сколько??
# Рандомно префикс
# Рандомно выбирается сотношение (это y) -> рандомно выбирается длина (непустая) -> рандомно выбираем слово этой длины (в итоге получим равномерное распределение по длинам и соотношениям)
# Рандомно выбираем другое соотношение (это n)
# Если nn, то также рандомно два соотношения, но ищем слово из фригруппы, которое не принадлежит


In [2]:
import numpy as np
from os import listdir
import random
import pickle
import time
from tqdm import tqdm
import func_timeout

In [3]:
from freegroup.tools import magnus_is_from_normal_closure

In [4]:
def to_string(word):
    return ' '.join(map(str, word))

In [5]:
def open_data(el):
    with open(el['path'], 'rb') as f:
        data = pickle.load(f)
    return data

In [5]:
datasets_dir = '/main/whitehead/data/datasets/L_100_data'
datasets_path = [
    dict(
        relation = tuple(map(int, el.split('_')[-1].split(','))),
        path = datasets_dir + '/' + el 
    ) for el in sorted(listdir(datasets_dir))
]

In [6]:
words_samples = {el['relation']: open_data(el) for el in datasets_path}
not_empty_lengths = {relation: [i for i in range(len(samples)) if samples[i]]
                     for relation, samples in words_samples.items()}

In [7]:
FREEGROUP_RELATION = (0,)
PREFIXES = [
    'nn', 'nn',
    'ny',
    'yn',
    ]
RELATION_LIST = list(map(lambda x: x['relation'], datasets_path))
TEST_RELATION_LIST = RELATION_LIST[1::4]
TRAIN_VAL_RELATION_LIST = [el for el in RELATION_LIST if el not in (TEST_RELATION_LIST + [FREEGROUP_RELATION])]


def choose_random_word_from_rel(rel):
    length = random.choice(not_empty_lengths[rel])
    word = random.choice(list(words_samples[rel][length]))
    return word, length

def choose_random_word_from_freegroup(rel1, rel2):
    while True:
        word, length = choose_random_word_from_rel(FREEGROUP_RELATION)
        if not (magnus_is_from_normal_closure(word, rel1) or magnus_is_from_normal_closure(word, rel2)):
            break
    return word, length

def choose_random_word_from_freegroup_time_limit(*args, **kwargs):
    TIMELIMIT = 0.3
    N_ATTEMPTS = 1
    for _ in range(N_ATTEMPTS):
        try:
            res =  func_timeout.func_timeout(
                    TIMELIMIT, choose_random_word_from_freegroup, args=args
                    )
            # print('func res', res)
            return res
        except:
            return None, None

def sample_raw_prompt(mode='train'):
    prefix = random.choice(PREFIXES)
    if mode == 'train':
        relation_list = TRAIN_VAL_RELATION_LIST
    else:
        relation_list = TEST_RELATION_LIST

    rel1 = random.choice(relation_list)
    rel2 = random.choice(list(set(relation_list).difference({rel1})))

    if prefix == 'yn':
        word, length = choose_random_word_from_rel(rel1)
    elif prefix == 'ny':
        word, length = choose_random_word_from_rel(rel2)
    elif prefix == 'nn':
        word, length = choose_random_word_from_freegroup_time_limit(rel1, rel2)
    else:
        raise ValueError
    return prefix, rel1, rel2, word, length

def process_raw_prompt(prefix, rel1, rel2, word, *args):
    if word is None:
        return None
    return f'{prefix}:{to_string(rel1)},{to_string(rel2)}:{to_string(word)}'

def collect_dataset_element(prefix, rel1, rel2, word, length):
    if word is None:
        return None
    return {
        'word': word,
        'word_str': to_string(word),
        'prompt': process_raw_prompt(prefix, rel1, rel2, word),
        'word_length': length,
        'relations': [rel1, rel2],
        'relation_lengths': [len(rel1), len(rel2)],
        'prefix': prefix
    }

In [ ]:
N_data = 500_000
from datetime import datetime
dataset = []
for i in range(N_data):
    element = collect_dataset_element(*sample_raw_prompt('train'))
    if element is not None:
        dataset.append(element)
    if i % 1000 == 0:
        with open('/main/whitehead/notebooks/log', 'a') as f:
            print(f'{i:07d}: {datetime.now()}: length={len(dataset)}', file=f)

with open('/main/whitehead/data/datasets/L_100_data_prompts_train_v2.pkl', 'wb') as f:
    pickle.dump(dataset, f)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f65c64ab9d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
func_timeout.dafunc.FunctionTimedOut-5561785348031968189: Function choose_random_word_from_freegroup (args=((-2, -2, -2, -3, 2, 2, 3, 3, -1), (-3, 2, -1, -2, 3, -1, 2, 1, 1, 3, -2, 1, -3, 2))) (kwargs={}) timed out after 0.300000 seconds.

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f65c64ab9d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
func_timeout.dafunc.FunctionTimedOut-8728858143536087501: Function choose_random_word_from_freegroup (args

In [ ]:
N_data = 10_000
from datetime import datetime
dataset_test = []
for i in range(N_data):
    element = collect_dataset_element(*sample_raw_prompt('test'))
    if element is not None:
        dataset_test.append(element)
    if i % 1000 == 0:
        with open('/main/whitehead/notebooks/log', 'a') as f:
            print(f'{i:07d}: {datetime.now()}: length={len(dataset_test)}', file=f)

with open('/main/whitehead/data/datasets/L_100_data_prompts_test_v2.pkl', 'wb') as f:
    pickle.dump(dataset_test, f)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f705585b9d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
  File "/usr/local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 790, in <setcomp>
    active_threads = {thread.ident for thread in threading.enumerate()}
func_timeout.dafunc.FunctionTimedOut3443246348419876346: Function choose_random_word_from_freegroup (args=((-1, -3, 2, 2, 2, 3, 2, 1), (2, -3, 2))) (kwargs={}) timed out after 0.300000 seconds.

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f7057d3a7a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):
func_timeout.dafunc.FunctionTimedOut-160619

In [203]:
# from tqdm import tqdm
# ans = []
# for _ in tqdm(range(1000)):
#     res = sample_raw_prompt()
#     ans.append(res[4])

In [200]:
len(list(filter(lambda x: x is not None, ans)))

437